In [1]:
'''
task: extract ontology terms and types from documents
model: gemma-2-2b-it
dataset: scholarly
evaluation: few-shot with prompting
'''
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# read data

import json

with open("/content/drive/MyDrive/Colab Notebooks/LLMs4OL/subtaskA/data/scholarly/train/documents.jsonl", "r") as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    print(f"result: {result}")
    print(isinstance(result, dict))
    print(result["text"])

result: {'id': '36_0', 'title': 'Temporal Qualifiers in Linguistics: Outdated, Obsolete, and Archaic Forms', 'text': 'In linguistics, certain forms are used to indicate that a word or expression is no longer in current use. These forms are classified as temporal qualifiers, which provide context about the timing or period when a particular linguistic expression was used. Specifically, the outdated form, obsolete form, and archaic form are all types of temporal qualifiers. The outdated form refers to a word or expression that was once commonly used but has since fallen out of favor. Similarly, the obsolete form is a word or expression that has completely gone out of use. The archaic form, on the other hand, is a word or expression that is no longer used in everyday language but may still be found in historical or literary contexts. All these forms serve as temporal qualifiers, helping to situate the language in a particular time or context.'}
True
In linguistics, certain forms are used 

In [3]:
# start preparing for QA pipeline
! pip install -U accelerate
! pip install -U transformers
!pip install transformers
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [4]:
import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
import nltk
import spacy
import string
import evaluate  # Bleu
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM

import warnings
warnings.filterwarnings("ignore")

In [5]:
# login to hugging face to have access to the model
!pip install huggingface_hub

from huggingface_hub import notebook_login

notebook_login()

In [6]:
# load zero-shot model
MODEL_NAME = "google/gemma-2-2b-it"
TOKENIZER = AutoTokenizer.from_pretrained(MODEL_NAME)
MODEL = AutoModelForCausalLM.from_pretrained(MODEL_NAME, return_dict=True)
DEVICE = "cuda:0"

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [7]:
#torch.cuda.empty_cache()
MODEL.to('cuda')

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm): Gemm

In [47]:
# create function to prepare few-shot examples (by default we use 3-shot)

with open('/content/drive/MyDrive/Colab Notebooks/LLMs4OL/subtaskA/data/scholarly/train/terms2docs.json', 'r') as json_data:
    terms_docs_json = json.load(json_data)

with open('/content/drive/MyDrive/Colab Notebooks/LLMs4OL/subtaskA/data/scholarly/train/terms2types.json', 'r') as json_data:
    term_types = json.load(json_data)

def find_term_from_doc_id(docid):
  terms = []
  for key in terms_docs_json:
    if (docid in terms_docs_json[key]):
        terms.append(key)
  return terms

def find_types_from_term(term):
  for elem in term_types:
    if elem['term'] == term:
        return elem['types']
  return []

def flatten(mylist):
  if not mylist:
    return []
  else:
    return [x for xs in mylist for x in xs]

def prepare_few_shot_prompt(json_list, num_few_shot=8):
  final_prompt = f"""
    <start_of_turn>user
    You are an expert ontologist. You are given a text between <INPUT></INPUT> tags and you need to extract all terms and types to create an ontology. A type is to a class what a term is to an object in object-oriented definition. Limit your output to 2 lists with keys 'terms' and 'types' and the values for each the list of terms and types respectively between <output> tags.
    """
  few_shots = json_list[:8]
  for json_str in few_shots:
    result = json.loads(json_str)
    document = result["text"]
    terms = find_term_from_doc_id(result["id"])
    types = []
    for term in terms:
      types.append(find_types_from_term(term))
    # flatten list of lists
    types = flatten(types)
    final_prompt += f"""<INPUT>{document}</INPUT>
    <end_of_turn>
    <start_of_turn>model
    <output>
    'terms:'{terms}
    'types:'{types}
    </output><end_of_turn>
    """
  return final_prompt

In [52]:
# evaluation metrics

import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
import re

def precision_at_k(actual, predicted):
    act_set = set(actual)
    pred_set = set(predicted)
    result = len(act_set & pred_set) / float(len(predicted))
    return result*100

def apk(actual, predicted, k):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if not actual:
        return 0.0
    if len(predicted)>k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i,p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    return score / min(len(actual), k)


def mapk(actual, predicted, k):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int,
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a, p, k) for a,p in zip(actual, predicted)])


class EvaluationMetrics:

    def __init__(self, ks:list, metric="map") -> None:
        self.ks = ks
        self.metric=metric

    def evaluate(self, actual:list, predicted:list):
        if self.metric == "map":
            return self.MAP(actual, predicted)
        else:
            return self.AP(actual, predicted)

    def MAP(self, actual:list, predicted:list):
        results_dict = {}
        for k in self.ks:
            results_dict["MAP@"+str(k)] = mapk(actual=actual, predicted=predicted, k=k)
        return results_dict

    def AP(self, actual:list, predicted:list):
        results_dict = {}
        for k in self.ks:
            results_dict["AP@"+str(k)] = [apk(actual=actual, predicted=predicted, k=k)
                                          for a, p in zip(actual, predicted)]
        return results_dict

def predict_answer(document, ref_relation=None):
    prompt = prepare_few_shot_prompt(json_list)
    inputs = TOKENIZER(prompt+f"""<DOCUMENT>{document}</DOCUMENT><end_of_turn>
    <start_of_turn>model""", return_tensors="pt").to(DEVICE)

    outputs = MODEL.generate(**inputs, max_new_tokens=120)

    predicted_relation = TOKENIZER.decode(outputs[0], skip_special_tokens=True)

    result = re.search('<output>(.*)</output>', predicted_relation)

    predicted_label = result.group(1) if result else "None"

    print("document: \n", document)
    print("predicted label: \n", predicted_relation)

    return predicted_relation

In [53]:
# test on one example
result = {"id": "10_1", "title": "Classification of Linguistic Elements: Pronouns, Adjectives, and Parts of Speech", "text": "In the realm of linguistics, various categories of words and phrases are identified based on their functions and characteristics. Pronouns, a fundamental part of speech, are further classified into several types, including weak personal pronouns, emphatic pronouns, allusive pronouns, strong personal pronouns, interrogative pronouns, fused pronoun auxiliaries, possessive relative pronouns, exclamative pronouns, reflexive personal pronouns, irreflexive personal pronouns, collective pronouns, reflexive possessive pronouns, affixed personal pronouns, and existential pronouns, all of which fall under the pronoun category. \n\nAdjectives, another crucial part of speech, are categorized into types such as qualifier adjectives, ordinal adjectives, past participle adjectives, adjective-i, adjective-na, present participle adjectives, possessive adjectives, and participle adjectives. Notably, qualifier adjectives are a subtype of adjective, highlighting the nuanced classifications within this part of speech.\n\nDeterminers, which are also a part of speech, include interrogative determiners, indefinite determiners, possessive determiners, reflexive determiners, demonstrative determiners, and relative determiners, showcasing the variety within this category.\n\nOther parts of speech include general adverbs, numerals (with subcategories like generic numerals and interrogative ordinal numerals), and numeral fractions. Additionally, elements like copulas, auxiliary verbs, infinitive particles, and coordinating conjunctions serve specific grammatical functions. Punctuation marks, such as colons, question marks, and open parentheses, are also considered parts of speech, playing vital roles in structuring written language.\n\nProper nouns and diminutive nouns represent specific types of nouns, while past participle adjectives and present participle adjectives illustrate the intersection between adjectives and verb forms. Circumpositions and reciprocal pronouns further expand the linguistic landscape, demonstrating the complexity and richness of language."}
predicted_answer = predict_answer(result['text'])
predicted_answer

document: 
 In the realm of linguistics, various categories of words and phrases are identified based on their functions and characteristics. Pronouns, a fundamental part of speech, are further classified into several types, including weak personal pronouns, emphatic pronouns, allusive pronouns, strong personal pronouns, interrogative pronouns, fused pronoun auxiliaries, possessive relative pronouns, exclamative pronouns, reflexive personal pronouns, irreflexive personal pronouns, collective pronouns, reflexive possessive pronouns, affixed personal pronouns, and existential pronouns, all of which fall under the pronoun category. 

Adjectives, another crucial part of speech, are categorized into types such as qualifier adjectives, ordinal adjectives, past participle adjectives, adjective-i, adjective-na, present participle adjectives, possessive adjectives, and participle adjectives. Notably, qualifier adjectives are a subtype of adjective, highlighting the nuanced classifications withi

'\n    user\n    You are an expert ontologist. You are given a text between <INPUT></INPUT> tags and you need to extract all terms and types to create an ontology. A type is to a class what a term is to an object in object-oriented definition. Limit your output to 2 lists with keys \'terms\' and \'types\' and the values for each the list of terms and types respectively between <output> tags.\n    <INPUT>In linguistics, certain forms are used to indicate that a word or expression is no longer in current use. These forms are classified as temporal qualifiers, which provide context about the timing or period when a particular linguistic expression was used. Specifically, the outdated form, obsolete form, and archaic form are all types of temporal qualifiers. The outdated form refers to a word or expression that was once commonly used but has since fallen out of favor. Similarly, the obsolete form is a word or expression that has completely gone out of use. The archaic form, on the other h

In [ ]:
# predict on all documents
for json_str in json_list:
    result = json.loads(json_str)
    predicted_answer = predict_answer(result["text"])